In [3]:
import os
os.chdir("C:\\Users\\tsarcevic\\PycharmProjects\\fingerprinting-toolbox")
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pickle
import numpy as np
import pandas as pd
import collections

from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from scipy.stats import chi2
from imblearn.under_sampling import *

from datasets import GermanCredit
from attacks import *
from parameter_guidelines.guidelines import *

In [5]:
# original data
data = Nursery().preprocessed()
X = data.drop('target', axis=1).to_numpy()
y = data['target']

In [6]:
# fingerprinted data
gamma=1
fp_dir = "parameter_guidelines/fingerprinted_data/nursery/attr_subset_8/"
fingerprinted_data = pd.read_csv(fp_dir + "universal_g{}_x1_l32_u1_sk0.csv".format(gamma))
fp_data = Nursery().preprocessed(fp_data=fingerprinted_data)
#data = data.to_numpy()
X_fp = fp_data.drop('target', axis=1)#.to_numpy()
y_fp = fp_data['target']

In [7]:
# baseline
score_baseline = fp_cross_val_score(LogisticRegression(random_state=9), X, y, X_fp, y_fp, scoring='accuracy')['test_score']
score_baseline

array([0.66743827, 0.75462963, 0.7804784 , 0.82747974, 0.64569664])

In [8]:
# this function will serve as a pipeline for cross validation attacked dataset
def attack_cross_val_score(X, y, target, attack, attack_strength, n_folds=5):
    accuracy = []
    for fold in range(n_folds):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=fold, shuffle=True)
        train = pd.concat([X_train, y_train], axis=1)
        attacked_train = attack.run(train, attack_strength, random_state=fold)
        attacked_X = attacked_train.drop(target, axis=1)
        attacked_y = attacked_train[target]
        model = LogisticRegression()
        model.fit(attacked_X, attacked_y)
        acc = accuracy_score(y_test, model.predict(X_test))
        accuracy.append(acc)
    return accuracy

In [9]:
# random attack
attack_utility_scores = attack_cross_val_score(X_fp, y_fp, 'target', HorizontalSubsetAttack(), 0.7)
attack_utility_scores

[0.7098765432098766,
 0.7040895061728395,
 0.7326388888888888,
 0.7087191358024691,
 0.7195216049382716]

### Targeted attack

In [10]:
def targeted_attack_cross_val_score(X, y, attack_strength, n_folds=5, sampler_version=1, sampler_n_neighbors=3):
    accuracy = []
    for fold in range(n_folds):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=fold, shuffle=True)
        train = pd.concat([X_train, y_train], axis=1)
        sampler = NearMiss(version=sampler_version, 
                           sampling_strategy=strategy(attack_strength, y_train), 
                           n_neighbors=sampler_n_neighbors)
        #sampler = InstanceHardnessThreshold(estimator=GradientBoostingClassifier(),
        #                                    sampling_strategy=strategy(attack_strength, y_train)) -- some internal error
        attacked_X, attacked_y = sampler.fit_resample(X_train, y_train)
        model = LogisticRegression()
        model.fit(attacked_X, attacked_y)
        acc = accuracy_score(y_test, model.predict(X_test))
        accuracy.append(acc)
    return accuracy

In [11]:
def strategy(attack_strength, y):
    strategy = y.value_counts() * (1-attack_strength)
    strategy = {key: round(x) for key, x in strategy.items()}
    return strategy

In [16]:
# targeted attack
attack_utility_scores = targeted_attack_cross_val_score(X_fp, y_fp, 0.7)
attack_utility_scores
np.mean(attack_utility_scores) # version 1 ###### the best (and the only one actually better than random)

0.5830246913580247

In [17]:
# experimenting with the number of neighbors
res = dict()
for n in range(1, 10):
    # targeted attack
    attack_utility_scores = targeted_attack_cross_val_score(X_fp, y_fp, 0.7, sampler_n_neighbors=n) # 3 shows to be the best (default)
    res[n] = np.mean(attack_utility_scores) # version 1 ###### the best (and the only one actually better than random)
print('version 1')
print(res)
print(max(res.values()))

version 1
{1: 0.6324845679012345, 2: 0.6084876543209876, 3: 0.5830246913580247, 4: 0.5804012345679013, 5: 0.5748456790123456, 6: 0.5712191358024692, 7: 0.5706790123456791, 8: 0.5688271604938271, 9: 0.5679783950617284}
0.6324845679012345


In [18]:
# targeted attack
attack_utility_scores = targeted_attack_cross_val_score(X_fp, y_fp, 0.7, sampler_version=2)
attack_utility_scores
np.mean(attack_utility_scores) # version 2

0.695216049382716

In [19]:
# targeted attack
attack_utility_scores = targeted_attack_cross_val_score(X_fp, y_fp, 0.7, sampler_version=3)
np.mean(attack_utility_scores) # version 3

0.5910493827160493

Here update the targeted_attack_cross_val_score with the best strategy.

In [20]:
with open('parameter_guidelines/Nursery/evaluation/robustness_horizontal_universal_c95_e30.pickle', 'rb') as infile:
    robustness = pickle.load(infile)

In [21]:
robustness

{1: 0.9,
 1.11: 0.9,
 1.25: 0.9,
 1.43: 0.9,
 1.67: 0.9,
 2: 0.9,
 2.5: 0.9,
 3: 0.9,
 4: 0.9,
 5: 0.9,
 10: 0.8,
 18: 0.6}

In [22]:
fpattr=8
n_exp = 10
results = {gamma: [] for gamma in robustness}
for gamma, attack_strength in robustness.items():
    print(gamma, attack_strength)
    for exp in range(n_exp):
        # take one fp ds
        fp_dir = "parameter_guidelines/fingerprinted_data/nursery/attr_subset_{}/".format(fpattr)
        fingerprinted_data = pd.read_csv(fp_dir + "universal_g{}_x1_l32_u1_sk{}.csv".format(gamma, exp))
        fp_data = Nursery().preprocessed(fp_data=fingerprinted_data)
        #data = data.to_numpy()
        X_fp = fp_data.drop('target', axis=1)#.to_numpy()
        y_fp = fp_data['target']
        
        # calc baseline fp utility
        score_baseline = fp_cross_val_score(LogisticRegression(random_state=9), X, y, X_fp, y_fp, scoring='accuracy')['test_score']

        # calc attacked 
        attack_utility_scores = targeted_attack_cross_val_score(X_fp, y_fp, attack_strength, 
                                                                sampler_version=2, sampler_n_neighbors=1) # CHANGE HERE

        # take relative loss
        rel_loss = (score_baseline - attack_utility_scores) / score_baseline
        results[gamma].append(rel_loss)

1 0.9
1.11 0.9
1.25 0.9
1.43 0.9
1.67 0.9
2 0.9
2.5 0.9
3 0.9
4 0.9
5 0.9
10 0.8
18 0.6


In [23]:
results

{1: [array([-0.00520231,  0.12730061,  0.12061295,  0.18175237, -0.06713147]),
  array([ 0.00289184,  0.13859383,  0.11890547,  0.20894843, -0.03725657]),
  array([ 0.0300578 ,  0.13403614,  0.14427367,  0.22019869, -0.00857667]),
  array([-0.00462428,  0.11015664,  0.12325349,  0.20283339, -0.00618668]),
  array([ 0.01729107,  0.1192053 ,  0.11967952,  0.25225346, -0.03546408]),
  array([0.07865818, 0.21084643, 0.2044665 , 0.28552471, 0.02966321]),
  array([ 0.06072874,  0.18315576,  0.15584416,  0.23778833, -0.01335665]),
  array([ 0.        ,  0.13394216,  0.1509716 ,  0.22133852, -0.01395415]),
  array([ 0.04279931,  0.15794872,  0.1440678 ,  0.23968459, -0.01156416]),
  array([ 0.01097631,  0.12531454,  0.10484274,  0.1865337 , -0.05159653])],
 1.11: [array([-0.0150289 ,  0.12022415,  0.10445545,  0.16839212, -0.07848394]),
  array([ 0.00578369,  0.12341932,  0.11050274,  0.19919641, -0.05518151]),
  array([ 0.03060046,  0.15813253,  0.14761431,  0.21741036, -0.02470911]),
  array

In [24]:
with open('parameter_guidelines/Nursery/evaluation/rel_undersampling_attack_utilities_lr_fpattr{}_e{}.pkl'.format(fpattr, n_exp), 'wb') as outfile:
    pickle.dump(results, outfile)